# Reissner-Mindlin plate
For thin three-dimensional plates, a dimension reduction is performed, transforming the 3D linear elasticity equations to equations on the two-dimensional surface $\Omega\subset\mathbb{R}^2$. The Reissner-Mindlin plate is a commonly used model; the vertical deflection $w$ and the linearized rotation vector $\beta$ are the unknowns. The total energy consists of the bending and shearing energy 
\begin{align*}
\mathcal{W}_{\mathrm{RM}}(w,\beta) = \frac{1}{2}\| \varepsilon(\beta) \|_{\mathbb{C},L_2}^2 + \frac{\kappa\,G}{2t^2} \| \nabla w - \beta \|^2_{L_2} - \int_{\Omega}f\cdot w\,dx,
\end{align*}
where $t$ denotes the thickness of the plate, $\varepsilon(\cdot)$ is the symmetric part of the gradient and the elasticity tensor is $\mathbb{C}\varepsilon=\frac{E}{12(1-\nu^2)}((1-\nu)\varepsilon+\nu\,\mathrm{tr}(\varepsilon)I_{2\times 2})$ (with $E$ Young's modulus, $\nu$ Poisson ratio). Further, $G= \frac{E}{2(1+\nu)}$ and $\kappa=5/6$ denote the shearing modulus and shear correction factor.

We are interested in positive, but small thicknesses $t>0$. Therefore, the shear energy acts as a penalty enforcing the equality $\beta = \nabla w$. If there are not enough finite element functions such that the discrete so-called Kirchhoff constraint can be satisfied, we observe bad results known as shear locking.

To avoid shear locking when the thickness $t$ becomes small several methods. We discuss two approaches:

* Mixed Interpolation of Tensorial Components (MITC)

* Rotations in Nèdèlec space using the Tangential-Displacement Normal-Normal-Stress method (TDNNS)

## Mixed Interpolation of Tensorial Components (MITC)
Direct approach where shear locking may occur
$$\frac{t^3}{12}\int_{\Omega} 2\mu\, \varepsilon(\beta):\varepsilon(\delta\beta) + \lambda\, \text{div}(\beta)\text{div}(\delta\beta)\,dx + t\kappa\,G\int_{\Omega}(\nabla u-\beta)\cdot(\nabla\delta u-\delta\beta)\,dx = \int_{\Omega} f\,\delta u\,dx,\qquad \forall (\delta u,\delta\beta). $$

Adding interpolation (reduction) operator $\boldsymbol{R}:[H^1_0(\Omega)]^2\to H(\text{curl})$. Spaces are chosen according to [<a href="http://math.aalto.fi/~rstenber/Publications/M3AS91.pdf">Brezzi, Fortin and Stenberg. Error analysis of mixed-interpolated elements for Reissner-Mindlin plates. <i>Mathematical Models and Methods in Applied Sciences 1</i>, 2
  (1991), 125-151.</a>]
$$\frac{t^3}{12}\int_{\Omega} 2\mu\, \varepsilon(\beta):\varepsilon(\delta\beta) + \lambda\, \text{div}(\beta)\text{div}(\delta\beta)\,dx + t\kappa\,G\int_{\Omega}\boldsymbol{R}(\nabla u-\beta)\cdot\boldsymbol{R}(\nabla\delta u-\delta\beta)\,dx = \int_{\Omega} f\,\delta u\,dx,\qquad \forall (\delta u,\delta\beta). $$

In [ ]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw

Set up benchmark example with exact solution. Young modulus $E$, Poisson ratio $\nu$, and shear correction factor $\kappa$.

In [ ]:
E, nu, k = 10.92, 0.3, 5 / 6
G = E / (2 * (1 + nu))
# force
fz = -1

def CMat(mat, E, nu):
    return E / (12 * (1 - nu**2)) * ((1 - nu) * mat + nu * Trace(mat) * Id(2))


def CMatInv(mat, E, nu):
    return  12*(1 - nu**2) / E * (1 / (1 - nu) * mat - nu / (1 - nu**2) * Trace(mat) * Id(2))

Due to symmetry we only need to mesh one quarter of the circle.

In [ ]:
R = 5

def GenerateMesh(order, maxh=1):
    circ = Circle((0, 0), R).Face()
    circ.edges.name = "circ"
    shape = (
        circ
        - MoveTo(-R, -R).Rectangle(R, 2 * R).Face()
        - MoveTo(-R, -R).Rectangle(2 * R, R).Face()
    )
    shape.edges.Min(X).name = "left"
    shape.edges.Min(Y).name = "bottom"
    return Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=maxh * R / 3)).Curve(order)


mesh = GenerateMesh(2)
Draw(mesh);

Depending on the boundary conditions (simply supported or clamped) we have different exact solutions for the vertical displacement. 

In [ ]:
def ExactSolution(t=0.1, clamped=True, draw=False):
    r = sqrt(x**2 + y**2)
    xi = r / R
    Db = E * t**3 / (12 * (1 - nu**2))

    if clamped:
        # exact solution for clamped bc
        w_ex = fz*t**3*R**4/(64 * Db)*(1 - xi**2)*((1 - xi**2) + 8*(t / R)**2/(3*k*(1 - nu)))
    else:
        # exact solution for simply supported bc
        w_ex = fz*t**3*R**4/(64*Db)*(1-xi**2)*((6+2*nu)/(1+nu)-(1+xi**2)+8*(t/R)**2/(3*k*(1-nu)))
    if draw:
        Draw(w_ex, mesh, "w_s_ex", deformation=True)
    return w_ex


ExactSolution(t=0.02, clamped=True, draw=True);

Use Lagrangian finite elements for rotations and the vertical deflection together with additional internal bubbles for order <4 for the rotations.

Adding interpolation (reduction) operator $\boldsymbol{R}:[H^1_0(\Omega)]^2\to H(\text{curl})$. Spaces are chosen according to [<a href="http://math.aalto.fi/~rstenber/Publications/M3AS91.pdf">Brezzi, Fortin and Stenberg. Error analysis of mixed-interpolated elements for Reissner-Mindlin plates. <i>Mathematical Models and Methods in Applied Sciences 1</i>, 2
  (1991), 125-151.</a>]
  
Find $(w,\beta)$ such that
$$\int_{\Omega} \mathbb{C}\varepsilon(\beta):\varepsilon(\delta) \,dx + \frac{\kappa\,G}{t^2}\int_{\Omega}\boldsymbol{R}(\nabla w-\beta)\cdot\boldsymbol{R}(\nabla v-\delta)\,dx = \int_{\Omega} f\,v\,dx,\qquad \forall (v,\delta). $$

In [ ]:
def Material(mat):
    return D * ((1 - nu) * mat + nu * Trace(mat) * Id(2))


def SolveRM(mesh, order=1, clamped=True, mitc=False, draw=False):
    if clamped:
        fesB = VectorH1(mesh, order=order, dirichletx="circ|left", dirichlety="circ|bottom")
    else:
        fesB = VectorH1(mesh, order=order, dirichletx="left", dirichlety="bottom")
    fesB.SetOrder(TRIG, order + 1 if order < 4 and order > 1 and mitc else order)
    fesB.Update()
    fesW = H1(mesh, order=order, dirichlet="circ")
    fes = fesB * fesW
    (beta, w), (delta, v) = fes.TnT()

    Gamma = HCurl(mesh, order=order, type1=True)

    a = BilinearForm(fes)
    a += InnerProduct(CMat(Sym(grad(beta)), E, nu), Sym(grad(delta)))*dx
    if mitc:
        if order > 1:
            a += k*G/t**2*Interpolate(grad(w)-beta, Gamma)*Interpolate(grad(v)-delta, Gamma)*dx
        else:
            # lowest order needs a stabilization technique
            # the stability parameter->0 for h->0
            h = specialcf.mesh_size
            a += k*G/(t**2+h**2)*Interpolate(grad(w)-beta,Gamma)*Interpolate(grad(v)-delta,Gamma)*dx
    else:
        a += k*G/t**2*(grad(w)-beta)*(grad(v)-delta)*dx

    f = LinearForm(fes)
    f += fz * v * dx

    gfsol = GridFunction(fes)
    gfbeta, gfw = gfsol.components

    with TaskManager():
        a.Assemble()
        f.Assemble()
        inv = a.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky")
        gfsol.vec.data = inv * f.vec
    if draw:
        Draw(gfw, mesh, "w")
        #Draw(Norm(Skew(Grad(gfbeta))), mesh)
    return gfw, fes

In [ ]:
results = []
clamped = False
ts = [1, 0.1, 0.01, 0.001]
order = 1  # 1,2

for t in ts:
    results.append([])
    w_ex = ExactSolution(t=t, clamped=clamped)
    norm_w = sqrt(Integrate(w_ex * w_ex, mesh))
    # Draw(w_ex, mesh)
    for i in range(5):
        mesh = GenerateMesh(order=order, maxh=0.5**i)
        gfw, fes = SolveRM(mesh, order=order, clamped=clamped, mitc=False, draw=False)

        err = sqrt(Integrate((gfw - w_ex) * (gfw - w_ex), mesh)) / norm_w
        results[-1].append((fes.ndof, err))

In [ ]:
import matplotlib.pyplot as plt

plt.yscale("log")
plt.xscale("log")
plt.xlabel("ndof")
plt.ylabel("relative error")
for i in range(len(results)):
    ndof, err = zip(*results[i])
    plt.plot(ndof, err, "-*", label="t=" + str(ts[i]))
plt.legend()
plt.show()

## TDNNS method

Next, we apply the TDNNS method to discretize the bending term $\| \varepsilon(\beta) \|_{\mathbb{C},L_2}^2$ as in [<a href="https://link.springer.com/article/10.1007/s00211-017-0883-9">Pechstein and Schöberl. The TDNNS method for Reissner-Mindlin plates. <i> Numerische Mathematik 137</i>, 3 (2017), 713-740</a>]. Instead of using Lagrangian elements together with an interpolation operator we we directly discretize the rotations $\beta$ in the Nedelec space. Thus, all discrete gradients $\nabla w$ are in the space of rotations, and we don't have the locking problem. By introducing the linearized bending moment $\sigma=\mathbb{C}\varepsilon(\beta)$ we obtain the following formulation.
<center>
<table><tr>
<td> <img src="hdivdiv_0_trig_2.png" width="150"/> </td>
<td> <img src="hcurl_type1_0_trig_1.png" width="120"/> </td>
<td> <img src="h1_p1_trig.png" width="120"/> </td>
</tr></table>
</center>

Find: $(\sigma,\beta, w) \in H(\mathrm{div} \mathrm{div})\times H(\mathrm{curl})\times H^1(\Omega)$ such that for all $(\tau,\delta, v) \in H(\mathrm{div} \mathrm{div})\times H(\mathrm{curl})\times H^1(\Omega)$
\begin{align*}
\begin{array}{ccccll}
-\int_{\Omega} \mathbb{C}^{-1} \sigma : \tau\,dx & - & \left< \mathrm{div} \tau, \beta \right> & = & 0 \\
-\left< \mathrm{div} \sigma, \delta \right> & + & \frac{\kappa\, G}{t^2} \int_{\Omega} (\nabla w - \beta) (\nabla v - \delta)\,dx & = & \int_{\Omega} f v\,dx,
\end{array}
\end{align*}
where
\begin{align*}
\langle \mathrm{div} \boldsymbol{\sigma}, \beta\rangle &= \sum_{T\in\mathcal{T}}\int_T\mathrm{div}\boldsymbol{\sigma}\cdot\beta\,dx -\int_{\partial T}\boldsymbol{\sigma}_{nt}\beta_t\,ds\\
&=-\sum_{T\in\mathcal{T}}\int_T\boldsymbol{\sigma}:\nabla \beta\,dx +\int_{\partial T}\boldsymbol{\sigma}_{nn}\beta_n\,ds =-\langle \boldsymbol{\sigma}, \nabla \beta\rangle
\end{align*}
and the inverted elasticity tensor (complience  tensor) $\mathbb{C}^{-1}\varepsilon = \frac{12(1-\nu^2)}{Et^3}\big(\frac{1}{1-\nu}\varepsilon-\frac{\nu}{1-\nu^2}\mathrm{tr}(\varepsilon)I_{2\times2}\big)$. Here $\partial T$ denotes the element-boundary (edges) of $T$, $n$ the normal vector and $t$ the tangential vector on $\partial T$. With e.g. $\sigma_{nt}:= t^\top\sigma n$ we denote the normal tangential component of $\sigma$.

In [ ]:
def MaterialInv(mat):
    return 1 / D * (1 / (1 - nu) * mat - nu / (1 - nu**2) * Trace(mat) * Id(2))


def SolveRM_TDNNS(mesh, order=1, clamped=True, draw=False):
    if clamped:
        fesB = HCurl(mesh, order=order - 1, dirichlet="circ")
        fesS = HDivDiv(mesh, order=order - 1, dirichlet="")
    else:
        fesB = HCurl(mesh, order=order - 1)
        fesS = HDivDiv(mesh, order=order - 1, dirichlet="circ")
    fesW = H1(mesh, order=order, dirichlet="circ")

    fes = fesW * fesB * fesS
    (w, beta, sigma), (v, delta, tau) = fes.TnT()

    n = specialcf.normal(2)

    a = BilinearForm(fes, symmetric=True)
    a += (-InnerProduct(CMatInv(sigma, E, nu), tau) + InnerProduct(tau, grad(beta)) \
        + InnerProduct(sigma, grad(delta)))*dx
    a += (-sigma[n,n]*(delta*n) - tau[n,n]*(beta*n))*dx(element_boundary=True)
    a += k*G/t**2*(grad(w) - beta)*(grad(v) - delta)*dx

    f = LinearForm(fes)
    f += fz * v * dx

    gfsol = GridFunction(fes)
    gfw, gfbeta, gfsigma = gfsol.components

    with TaskManager():
        a.Assemble()
        f.Assemble()
        inv = a.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky")
        gfsol.vec.data = inv * f.vec
    if draw:
        Draw(gfw, mesh, "w")

    return gfw, fes

In [ ]:
results_tdnns = []
clamped = True

order = 1
ts = [1, 0.1, 0.01, 0.001]
for t in ts:
    results_tdnns.append([])
    w_ex = ExactSolution(t=t, clamped=clamped)
    norm_w = sqrt(Integrate(w_ex * w_ex, mesh))

    for i in range(5):
        mesh = GenerateMesh(order=order, maxh=0.5**i)
        gfw, fes = SolveRM_TDNNS(mesh, order=order, clamped=clamped, draw=False)

        err = sqrt(Integrate((gfw - w_ex) * (gfw - w_ex), mesh)) / norm_w
        results_tdnns[-1].append((fes.ndof, err))

In [ ]:
import matplotlib.pyplot as plt

plt.yscale("log")
plt.xscale("log")
plt.xlabel("ndof")
plt.ylabel("relative error")
for i in range(len(results_tdnns)):
    ndof, err = zip(*results_tdnns[i])
    plt.plot(ndof, err, "-*", label="t=" + str(ts[i]))
plt.legend()
plt.show()

In the limit $t \to 0$ the shear energy term can be understood as a penalty formulation enforcing the Kirchhoff-Love assumption
\begin{align*}
\beta = \nabla w.
\end{align*}
Thus, in the limit case, the total energy simplifies by eliminating the rotation $\beta$ to
\begin{align*}
\mathcal{W}_{\mathrm{KL}}(w)=\frac{1}{2}\| \varepsilon(\nabla w) \|_{\mathbb{C},L_2}^2-\int_{\Omega}f\,w,
\end{align*}
which is the Kirchhoff-Love plate model.

Drawback of the TDNNS formulation: It yields a mixed saddle point problem. We can apply hybridization techniques by breaking the normal-normal continuity of $\sigma$ and reinforcing it by an additional Lagrange multiplier $\alpha$. A symmetric positive definite system is regained after performing static condensation and eliminating $\sigma$ at the element level.

Exercise:
* Implement hybridization for TDNNS